In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras import layers
from sklearn import datasets
from tornado.options import define

from data_processing import data_processing
from scipy.optimize import linprog

## Fairness pre-specified Level

In [2]:
g=0.01
l=0.01
m=0.01                      

## Data Process

In [ ]:
train_features_1, vali_features_1, test_features_1, train_labels_1, vali_labels_1, test_labels_1, train_sensitive_1, vali_sensitive_1, test_sensitive_1,train_features_2,vali_features_2, test_features_2, train_labels_2, vali_labels_2, test_labels_2, train_sensitive_2, vali_sensitive_2, test_sensitive_2=data_processing()



## Train Model by FedAvg

In [4]:
def create_model(input_shape):
    model = tf.keras.models.Sequential([
        tf.keras.layers.InputLayer(input_shape=input_shape),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
input_shape = train_features_1.shape[1:]  
global_model = create_model(input_shape)
global_model.load_weights('Fed_Avg.h5')


## Evaluations of Model Trained By FedAvg

In [6]:

def compute_demographic_parity(y_true, y_pred, groups):
    y_true = np.asarray(y_true).flatten()
    y_pred = np.asarray(y_pred).flatten()
    groups = np.asarray(groups).flatten()
    white_pr = np.mean(y_pred[groups == 1])
    black_pr = np.mean(y_pred[groups == 0])
    DP=abs(white_pr-black_pr)
    return DP
def compute_accuracy(y_true, y_pred):
    y_true = np.asarray(y_true).flatten()
    y_pred = np.asarray(y_pred).flatten()
    accuracy = np.mean(y_true == y_pred)
    return accuracy

In [ ]:
local_SP=[]
for client_data in [(test_features_1, test_labels_1, test_sensitive_1), (test_features_2, test_labels_2, test_sensitive_2)]:
    vali_features, vali_labels,vali_sensitive = client_data
    y_val_pred=global_model.predict(vali_features)
    y_val_pred=np.where(y_val_pred<0.5,0,1)
    DP=compute_demographic_parity(vali_labels,y_val_pred,vali_sensitive)
    local_SP.append(DP)
local_SP_avg_1=np.mean(local_SP)
global_features=np.concatenate((test_features_1,test_features_2),axis=0)
global_labels=np.concatenate((test_labels_1,test_labels_2),axis=0)
global_sensitive=np.concatenate((test_sensitive_1,test_sensitive_2),axis=0)
y_global_pred=global_model.predict(global_features)
y_global_pred=np.where(y_global_pred<0.5,0,1)
global_SP_1=compute_demographic_parity(global_labels,y_global_pred,global_sensitive)
y_global_pred=y_global_pred.flatten()
global_labels=global_labels.flatten()
acc_fed=compute_accuracy(global_labels,y_global_pred)

y_1_pred=global_model.predict(test_features_1)  
y_1_pred=np.where(y_1_pred<0.5,0,1)
y_1_pred=y_1_pred.flatten()
acc_1=compute_accuracy(test_labels_1,y_1_pred)
y_2_pred=global_model.predict(test_features_2)
y_2_pred=np.where(y_2_pred<0.5,0,1)
y_2_pred=y_2_pred.flatten()
acc_2=compute_accuracy(test_labels_2,y_2_pred)
client_disparity_1=abs(acc_1-acc_2)

print('###################################')
print('local disparity',local_SP_avg_1)
print('global disparity',global_SP_1)
print('client disparity',client_disparity_1)
print('global accuracy',acc_fed)


## Compute the statistics for fair LP

In [8]:

## p(y,a|c)
def p_computation(label, sensitive_attribute, y, a):
    label=np.reshape(label,(-1,))
    N=len(label)
    sensitive_attribute=np.reshape(sensitive_attribute,(-1,))# Total number of samples
    # Create a boolean mask for where both conditions are met
    mask = (label == y) & (sensitive_attribute == a)
    # print(mask)
    # print("Number of matching samples:", np.sum(mask))
    
    # Compute the probability
    p_y_ac = np.sum(mask) / N
    return p_y_ac
#community_1
## p(y,a,c)
S= np.zeros((2,2,2))
p=[len(vali_features_1)/(len(vali_features_1)+len(vali_features_2)),len(vali_features_2)/(len(vali_features_1)+len(vali_features_2))]
for c, (feature, label, sensitive) in enumerate([(vali_features_1, vali_labels_1, vali_sensitive_1), (vali_features_2, vali_labels_2,vali_sensitive_2)]):
    for a in range(2):
        for y in range(2):
            S[c,a,y]=p[c]*p_computation(label,sensitive,y,a)
alpha_ac=np.sum(S,axis=-1)


L=np.zeros((2,2,2))
for c, (feature, label, sensitive) in enumerate([(vali_features_1, vali_labels_1, vali_sensitive_1), (vali_features_2, vali_labels_2,vali_sensitive_2)]):
    for a in range(2):
        for y in range(2):
            L[c,a,y]=p_computation(label,sensitive,y,a)



[[[0.29027511 0.03525272]
  [0.46628279 0.19603327]]

 [[0.00108765 0.00089571]
  [0.00204734 0.0081254 ]]]
1.0
[[0.32552783 0.66231606]
 [0.00198337 0.01017274]]
[[[0.29384715 0.03568653]
  [0.47202073 0.1984456 ]]

 [[0.08947368 0.07368421]
  [0.16842105 0.66842105]]]


In [ ]:
## TP of the optimal predictor

TP= np.zeros((2,2,2))
def compute_TP (y_pred,label, sensitive_attribute, y, a):
    y_pred=np.reshape(y_pred,(-1,))
    label=np.reshape(label,(-1,))
    N=len(y_pred)
    sensitive_attribute=np.reshape(sensitive_attribute,(-1,))# Total number of samples
    # Create a boolean mask for where both conditions are met
    count_1= (y_pred == y) & (label == y) & (sensitive_attribute == a)
    count_2 = (label== y) & (sensitive_attribute == a)
    print(np.sum(count_1),np.sum(count_2))
    
    # Compute the probability
    TP_y_ac = np.sum(count_1) / np.sum(count_2)
    return TP_y_ac
for c, (feature, label, sensitive) in enumerate([(vali_features_1, vali_labels_1, vali_sensitive_1), (vali_features_2, vali_labels_2,vali_sensitive_2)]):
    y_pred=global_model.predict(feature)
    y_pred=np.where(y_pred<0.5,0,1)
    for a in range(2):
        for y in range(2):
            TP[c,a,y]=compute_TP(y_pred,label,sensitive,y,a)
print(TP)

In [ ]:
def compute_alpha(label, sensitive_attribute, y, a):
    label=np.reshape(label,(-1,))
    N=len(label)
    sensitive_attribute=np.reshape(sensitive_attribute,(-1,))# Total number of samples
    # Create a boolean mask for where both conditions are met
    mask = (label == y) & (sensitive_attribute == a)
    # print(mask)
    # print("Number of matching samples:", np.sum(mask))
    
    # Compute the probability
    alpha_y_ac = np.sum(mask) / N
    return alpha_y_ac
alpha_a_y = np.zeros((2,2))
for a in range(2):
    for y in range(2):
        alpha_a_y[a,y]=compute_alpha(global_labels,global_sensitive,y,a)
print(alpha_a_y)
def compute_e(sensitive,a):
    sensitive=np.reshape(sensitive,(-1,))
    N=len(sensitive)
    mask = (sensitive == a)
    e_a = np.sum(mask) / N
    return e_a
e_a=np.zeros(2)
for a in range(2):
    e_a[a]=compute_e(global_sensitive,a)



## LP for statistical Parity

In [ ]:
def LP_DP(e_g,e_l,e_c):
# define the objective function
    C=[]
    for c in range(2):
        for a in range(2):
            for y in range(2):
                C.append(-S[c,a,y])

    #define the global fairness constraints



    ## global fairness constraints
    A_1=[-S[0,0,0]/e_a[0],S[0,0,1]/e_a[0],S[0,1,0]/e_a[1],-S[0,1,1]/e_a[1], -S[1,0,0]/e_a[0],S[1,0,1]/e_a[0], S[1,1,0]/e_a[1],-S[1,1,1]/e_a[1]]

    b_1=[-S[0,0,0]/e_a[0]+S[0,1,0]/e_a[1]-S[1,0,0]/e_a[0]+S[1,1,0]/e_a[1]]


    ## local fairness constraints
    A_2=[-S[0,0,0]/alpha_ac[0,0],S[0,0,1]/alpha_ac[0,0],S[0,1,0]/alpha_ac[0,1],-S[0,1,1]/alpha_ac[0,1],0,0,0,0]

    b_2=[-S[0,0,0]/alpha_ac[0,0]+S[0,1,0]/alpha_ac[0,1]]


    A_3=[0,0,0,0,-S[1,0,0]/alpha_ac[1,0],S[1,0,1]/alpha_ac[1,0],S[1,1,0]/alpha_ac[1,1],-S[1,1,1]/alpha_ac[1,1]]

    b_3=[-S[1,0,0]/alpha_ac[1,0]+S[1,1,0]/alpha_ac[1,1]]
    ## client fairness constraints
    A_4=[L[0,0,0],L[0,0,1],L[0,1,0],L[0,1,1],-L[1,0,0],-L[1,0,1],-L[1,1,0],-L[1,1,1]]
    b_4=[0]
    A_1=np.array(A_1)
    A_2=np.array(A_2) 
    A_3=np.array(A_3)
    A_4=np.array(A_4)
    
    print(A_4)


    ## constraints for the feasible region of LP
    def K_ac_compute(a,c):
        K_ac=np.zeros((3,2))
        l_ac=np.zeros((3,1))
        K_ac[0:]=[-1,-1]
        K_ac[1:]=[(1-TP[c,a,1]), TP[c,a,0]]
        K_ac[2:]=[TP[c,a,1], (1-TP[c,a,0])]
        l_ac[0:]=[-1]
        l_ac[1:]=[TP[c,a,0]]
        l_ac[2:]=[TP[c,a,1]]
        return K_ac,l_ac
    
    # Define the submatrices k_01, k_11, k_02, k_12 as 3x2 matrices
    k_01 ,l_01= K_ac_compute(0,0)
    k_11 ,l_11= K_ac_compute(1,0)
    k_02 ,l_02= K_ac_compute(0,1)
    k_12 ,l_12= K_ac_compute(1,1)
    
    
    # Initialize the large matrix with zeros
    M = np.zeros((12, 8))  # 4 blocks of 3x2 matrices; resulting in a 12x8 matrix
    l=np.zeros((12,1))
    # Place each submatrix on the diagonal
    M[0:3, 0:2] = k_01  # Place k_01 in the top-left
    M[3:6, 2:4] = k_11  # Place k_11 in the next diagonal block
    M[6:9, 4:6] = k_02  # Place k_02 in the next diagonal block
    M[9:12, 6:8] = k_12 # Place k_12 in the bottom-right block
    
    l[0:3]=l_01
    l[3:6]=l_11
    l[6:9]=l_02
    l[9:12]=l_12
    # print(np.shape(A_1),np.shape(A_2),np.shape(A_3),np.shape(M))
    # combine the constraints
    A=np.vstack((A_1,A_2,A_3,A_4, -A_1, -A_2, -A_3,-A_4,M))
    trail_1=[e_g]
    trail_2=[e_l]
    trail_3=[e_c]
    b_1=np.reshape(b_1,(1,1))
    b_2=np.reshape(b_2,(1,1))
    b_3=np.reshape(b_3,(1,1))
    b_4=np.reshape(b_4,(1,1))
    # print(A)
    # print(b_local)
    b_r=np.vstack((trail_1,trail_2,trail_2,trail_3, trail_1,trail_2,trail_2,trail_3, np.zeros((12,1))))
    b_t=np.vstack((b_1,b_2,b_3,b_4,-b_1,-b_2,-b_3,-b_4,l))
    b=b_r+b_t
    # print(b)
    res = linprog(C, A_ub=A, b_ub=b)
    x=res.x
    l=res.fun
    x=np.reshape(x,(len(x),1))
    x=np.reshape(x,(2,2,2))
    return x
x=LP_DP(g,l,m)



## LP for Equalized Odds

In [12]:
# def LP_EO(e_0,e_c):
# # define the objective function
#     C=[]
#     for c in range(2):
#         for a in range(2):
#             for y in range(2):
#                 C.append(-S[c,a,y])
#     #define the global fairness constraints
#     N=2
#     K=2
#     A_1=[]
#     A_2=[]
#     for c in range(2):
#         for a in range(2):
#             for y in range(2):
#                 if  a==0 and y==0:
#                     A_1.append(-S[c,a,y]/alpha_a_y[a,y])
#                     # print(S[c,a,y],alpha_a_y[a,y])
#                 elif  a==1 and y==0:
#                     A_1.append(S[c,a,y]/alpha_a_y[a,y])
    
#                 else:
#                     A_1.append(0)
#                     print(2)
    
#     for c in range(2):
#         for a in range(2):
#             for y in range(2):
#                 if c==c and a==0 and y==1:
#                     A_2.append(-S[c,a,y]/alpha_a_y[a,y])
#                 elif c==c and a==1 and y==1:
#                     A_2.append(S[c,a,y]/alpha_a_y[a,y])
#                 else:
#                     A_2.append(0)
#     # define local fairness constraints
#     basis_vectors = np.eye(N)  # Identity matrix of size 50x50, where each row is a basis vector
#     A_1=np.array(A_1)
#     A_2=np.array(A_2)
#     # print(np.shape(A_1),np.shape(A_2))
#     # To access e_i, you can use basis_vectors[i], e.g., e_0 is basis_vectors[0]
    
#     # Define zero vector in R^50
#     zero_vec = np.zeros((N,N))
    
    
#     # Construct the matrix
#     # First row: [e_0, e_1, 0, 0]
#     row1 = np.hstack((basis_vectors, -basis_vectors, zero_vec, zero_vec))
    
#     # Second row: [0, 0, e_0, e_1]
#     row2 = np.hstack((zero_vec, zero_vec, basis_vectors, -basis_vectors))
    
#     # Combine rows into a single matrix
#     A_3 = np.vstack((row1, row2))
#     # define the property of dervied outcome predictor
#     def K_ac_compute(a,c):
#         K_ac=np.zeros((3,2))
#         l_ac=np.zeros((3,1))
#         K_ac[0:]=[-1,-1]
#         K_ac[1:]=[(1-TP[c,a,1]), TP[c,a,0]]
#         K_ac[2:]=[TP[c,a,1], (1-TP[c,a,0])]
#         l_ac[0:]=[-1]
#         l_ac[1:]=[TP[c,a,0]]
#         l_ac[2:]=[TP[c,a,1]]
#         return K_ac,l_ac
    
#     # Define the submatrices k_01, k_11, k_02, k_12 as 3x2 matrices
#     k_01 ,l_01= K_ac_compute(0,0)
#     k_11 ,l_11= K_ac_compute(1,0)
#     k_02 ,l_02= K_ac_compute(0,1)
#     k_12 ,l_12= K_ac_compute(1,1)
    
    
#     # Initialize the large matrix with zeros
#     M = np.zeros((12, 8))  # 4 blocks of 3x2 matrices; resulting in a 12x8 matrix
#     l=np.zeros((12,1))
#     # Place each submatrix on the diagonal
#     M[0:3, 0:2] = k_01  # Place k_01 in the top-left
#     M[3:6, 2:4] = k_11  # Place k_11 in the next diagonal block
#     M[6:9, 4:6] = k_02  # Place k_02 in the next diagonal block
#     M[9:12, 6:8] = k_12 # Place k_12 in the bottom-right block
    
#     l[0:3]=l_01
#     l[3:6]=l_11
#     l[6:9]=l_02
#     l[9:12]=l_12
#     # print(np.shape(A_1),np.shape(A_2),np.shape(A_3),np.shape(M))
#     # combine the constraints
#     A=np.vstack((A_1,A_2,A_3, -A_1, -A_2, -A_3,M))
#     # print(A)
#     b_global=e_0*np.ones((2,1))
#     b_local=e_c*np.ones((4,1))
#     # print(b_local)
#     b=np.vstack((b_global,b_local,b_global,b_local,l))
#     # print(b)
#     res = linprog(C, A_ub=A, b_ub=b)
#     x=res.x
#     x=np.reshape(x,(2,2,2))
#     print(res.fun)
#     return x

## LP for equal opportunity

In [13]:
# def LP_EOp(e_0,e_c):
# # define the objective function
#     C=[]
#     for c in range(2):
#         for a in range(2):
#             for y in range(2):
#                 C.append(-S[c,a,y])
#     #define the global fairness constraints
#     N=2
#     K=2
#     A_2=[]

    
#     for c in range(2):
#         for a in range(2):
#             for y in range(2):
#                 if c==c and a==0 and y==1:
#                     A_2.append(-S[c,a,y]/alpha_a_y[a,y])
#                 elif c==c and a==1 and y==1:
#                     A_2.append(S[c,a,y]/alpha_a_y[a,y])
#                 else:
#                     A_2.append(0)
#     # define local fairness constraints
#     basis_vectors = [[0,1]]
#     basis_vectors=np.reshape(basis_vectors,(1,2))# Identity matrix of size 50x50, where each row is a basis vector
#     A_2=np.array(A_2)
#     # print(np.shape(A_1),np.shape(A_2))
#     # To access e_i, you can use basis_vectors[i], e.g., e_0 is basis_vectors[0]
    
#     # Define zero vector in R^50
#     zero_vec = np.zeros((1,2))
    
    
#     # Construct the matrix
#     # First row: [e_0, e_1, 0, 0]
#     row1 = np.hstack((basis_vectors, -basis_vectors, zero_vec, zero_vec))
    
#     # Second row: [0, 0, e_0, e_1]
#     row2 = np.hstack((zero_vec, zero_vec, basis_vectors, -basis_vectors))
    
#     # Combine rows into a single matrix
#     A_3 = np.vstack((row1, row2))
#     # define the property of dervied outcome predictor
#     def K_ac_compute(a,c):
#         K_ac=np.zeros((3,2))
#         l_ac=np.zeros((3,1))
#         K_ac[0:]=[-1,-1]
#         K_ac[1:]=[(1-TP[c,a,1]), TP[c,a,0]]
#         K_ac[2:]=[TP[c,a,1], (1-TP[c,a,0])]
#         l_ac[0:]=[-1]
#         l_ac[1:]=[TP[c,a,0]]
#         l_ac[2:]=[TP[c,a,1]]
#         return K_ac,l_ac
    
#     # Define the submatrices k_01, k_11, k_02, k_12 as 3x2 matrices
#     k_01 ,l_01= K_ac_compute(0,0)
#     k_11 ,l_11= K_ac_compute(1,0)
#     k_02 ,l_02= K_ac_compute(0,1)
#     k_12 ,l_12= K_ac_compute(1,1)
    
    
#     # Initialize the large matrix with zeros
#     M = np.zeros((12, 8))  # 4 blocks of 3x2 matrices; resulting in a 12x8 matrix
#     l=np.zeros((12,1))
#     # Place each submatrix on the diagonal
#     M[0:3, 0:2] = k_01  # Place k_01 in the top-left
#     M[3:6, 2:4] = k_11  # Place k_11 in the next diagonal block
#     M[6:9, 4:6] = k_02  # Place k_02 in the next diagonal block
#     M[9:12, 6:8] = k_12 # Place k_12 in the bottom-right block
    
#     l[0:3]=l_01
#     l[3:6]=l_11
#     l[6:9]=l_02
#     l[9:12]=l_12
#     # print(np.shape(A_1),np.shape(A_2),np.shape(A_3),np.shape(M))
#     # combine the constraints
#     A=np.vstack((A_2,A_3, -A_2, -A_3,M))
#     # print(A)
#     b_global=e_0*np.ones((1,1))
#     b_local=e_c*np.ones((2,1))
#     # print(b_local)
#     b=np.vstack((b_global,b_local,b_global,b_local,l))
#     # print(b)
#     res = linprog(C, A_ub=A, b_ub=b)
#     x=res.x
#     x=np.reshape(x,(2,2,2))
#     print(res.fun)
#     return x
# x=LP_EOp(0.0001,0.0001)


## Construct Fair Outcome Prediction

In [ ]:
beta=np.zeros((2,2,3))
for a in range(2):
        for c in range(2):
            A=np.array([[1,1,1],[TP[c,a,0],1,0],[TP[c,a,1],0,1]])
            b=np.array([1,x[c,a,0],x[c,a,1]])
            beta_ac=np.linalg.solve(A,b)
            beta[c,a,:]=beta_ac
beta=np.reshape(beta,(2,2,3))
# print(beta)

def compute_tilde_Y(c,a, Y_hat, y_values, beta):
    """
    Compute \widetilde{Y}_{\boldsymbol{\beta}_{ac}}(x,a,c) based on given probabilities.

    Parameters:
    - beta_ac: Dictionary with keys 'beta_0' and 'beta_y' for probabilities.
    - Y_hat: The predicted value \hat{Y}(x,a,c).
    - y_values: List or array of possible y values in \mathcal{Y}.

    Returns:
    - tilde_Y: The computed value of \widetilde{Y}.
    """

    beta_0 = beta[c, a, 0]  # Probability for Y_hat
    beta_y = beta[c, a, 1:] 
    # Probabilities for other y in \mathcal{Y}
    
    # Normalize probabilities
    
    # Generate a random number
    rand_val = np.random.rand()
    
    # Determine the output based on random value
    if rand_val < beta_0:
        return Y_hat
    else:
        cumulative_prob = beta_0
        for y in y_values:
            cumulative_prob += beta_y[y]
            if rand_val < cumulative_prob:
                return y
y_tilde_1=[]
y_tilde_2=[]
# Example usage
for c, (feature, label, sensitive) in enumerate([(vali_features_1, vali_labels_1, vali_sensitive_1), (vali_features_2, vali_labels_2,vali_sensitive_2)]):
    y_pred=global_model.predict(feature)
    y_pred=np.where(y_pred<0.5,0,1)
    for i in range (len(y_pred)):
        a=int(sensitive[i])
        y_hat=y_pred[i]
        y_tilde=compute_tilde_Y(c,a,y_hat,[0,1],beta)
        y_tilde=int(y_tilde)
        if c==0:
            y_tilde_1.append(y_tilde)
        elif c==1:
            y_tilde_2.append(y_tilde)

In [ ]:
y_tilde_1=[]
y_tilde_2=[]
# Example usage
for c, (feature, label, sensitive) in enumerate([(test_features_1, test_labels_1, test_sensitive_1), (test_features_2, test_labels_2,test_sensitive_2)]):
    y_pred=global_model.predict(feature)
    y_pred=np.where(y_pred<0.5,0,1)
    for i in range (len(y_pred)):
        a=int(sensitive[i])
        y_hat=y_pred[i]
        y_tilde=compute_tilde_Y(c,a,y_hat,[0,1],beta)
        y_tilde=int(y_tilde)
        if c==0:
            y_tilde_1.append(y_tilde)
        elif c==1:
            y_tilde_2.append(y_tilde)

In [16]:
local_SP=[]
post_SP_1=compute_demographic_parity(test_labels_1,y_tilde_1,test_sensitive_1)
local_SP.append(post_SP_1)
post_SP_2=compute_demographic_parity(test_labels_2,y_tilde_2,test_sensitive_2)
local_SP.append(post_SP_2)
print('local SP',local_SP)
local_SP_avg=np.mean(local_SP)
print('local SP',local_SP_avg)
y_tilde=np.concatenate((y_tilde_1,y_tilde_2),axis=0)
global_test_label=np.concatenate((test_labels_1,test_labels_2),axis=0)
global_test_sensitive=np.concatenate((test_sensitive_1,test_sensitive_2),axis=0)
post_SP=compute_demographic_parity(global_test_label,y_tilde,global_test_sensitive)
print('global SP', post_SP)
acc_count=np.where(y_tilde==global_test_label,1,0)
acc=np.sum(acc_count)/len(acc_count)
print('global acc',acc)
y_tilde_1=np.array(y_tilde_1)
y_tilde_2=np.array(y_tilde_2)
acc_1=compute_accuracy(test_labels_1,y_tilde_1)
acc_2=compute_accuracy(test_labels_2,y_tilde_2)
client_dis=abs(acc_1-acc_2)
print('client acc',acc_1,acc_2)
print('client disparity',client_dis)

local SP [0.33144700843688624, 0.375]
local SP 0.3532235042184431
global SP 0.33587339572525327
global acc 0.7936329204626881
client acc 0.7934715025906736 0.8067226890756303
client disparity 0.013251186484956712


## before post-processing

In [ ]:
print('local SP',local_SP_avg_1)
print('global SP',global_SP_1)
print('global acc',acc_fed)
print('client disparity',client_dis)

## after post-processing

In [ ]:
print('local SP',local_SP_avg) 
print('global SP',post_SP)
print('global acc',acc)
print('client disparity', client_dis)